In [19]:
# Print successors example
print_succ([4,3,0,5,1,6,7,2,0])

# Solve puzzle example
solve([4,3,0,5,1,9,7,2,0])

[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6
[4, 3, 0, 5, 1, 0, 7, 2, 6] h=8
[4, 3, 0, 5, 1, 6, 7, 0, 2] h=8
[4, 3, 6, 5, 1, 0, 7, 2, 0] h=8
True


ValueError: 9 is not in list

In [22]:
import heapq

def is_valid_state(state):
    """
    Check if state follows the rules:
    - No duplicates
    - Continuous integers starting from 1
    - No skipped integers
    """
    # Get non-zero numbers
    numbers = [x for x in state if x != 0]
    
    # Check for duplicates
    if len(numbers) != len(set(numbers)):
        return False
        
    # Check for continuous integers starting from 1
    max_num = max(numbers)
    expected_numbers = set(range(1, max_num + 1))
    if set(numbers) != expected_numbers:
        return False
        
    return True

def get_manhattan_distance(state):
    """Calculate total Manhattan distance for all tiles to their goal positions."""
    distance = 0
    n = len([x for x in state if x != 0])  # Count non-zero tiles
    goal_state = list(range(1, n + 1)) + [0] * (9 - n)
    
    for i in range(9):
        if state[i] != 0:  # Only calculate for non-zero tiles
            curr_row, curr_col = i // 3, i % 3
            goal_idx = goal_state.index(state[i])
            goal_row, goal_col = goal_idx // 3, goal_idx % 3
            distance += abs(curr_row - goal_row) + abs(curr_col - goal_col)
    return distance

def get_successors(state):
    """Generate all possible successor states from current state."""
    successors = []
    # Find indices of empty spaces
    zero_indices = [i for i, val in enumerate(state) if val == 0]
    
    for zero_idx in zero_indices:
        row, col = zero_idx // 3, zero_idx % 3
        
        # Check possible moves (up, down, left, right)
        moves = []
        if row > 0: moves.append(-3)  # up
        if row < 2: moves.append(3)   # down
        if col > 0: moves.append(-1)  # left
        if col < 2: moves.append(1)   # right
        
        for move in moves:
            new_idx = zero_idx + move
            # Make sure new position is valid and not another empty space
            if (0 <= new_idx < 9 and 
                state[new_idx] != 0 and
                (move in [-3, 3] or new_idx // 3 == zero_idx // 3)):
                new_state = list(state)
                new_state[zero_idx], new_state[new_idx] = new_state[new_idx], new_state[zero_idx]
                successors.append(new_state)
    
    return sorted(successors)

def print_succ(state):
    """Print all successor states with their heuristic values."""
    if not is_valid_state(state):
        return
    successors = get_successors(state)
    for succ in successors:
        h = get_manhattan_distance(succ)
        print(f"{succ} h={h}")

def solve(state):
    """Solve the puzzle using A* search algorithm."""
    # First check if state is valid
    if not is_valid_state(state):
        print(False)
        return
    
    print(True)
    
    # Initialize variables
    n = len([x for x in state if x != 0])  # Count non-zero tiles
    goal_state = list(range(1, n + 1)) + [0] * (9 - n)
    initial_h = get_manhattan_distance(state)
    
    # Priority queue entries: (f_score, move_count, state, path)
    pq = [(initial_h, 0, state, [])]
    visited = set()
    max_queue_length = 1
    
    while pq:
        max_queue_length = max(max_queue_length, len(pq))
        f_score, moves, current_state, path = heapq.heappop(pq)
        
        # Add current state to path
        current_path = path + [current_state]
        
        # Print current state
        print(f"{current_state} h={get_manhattan_distance(current_state)} moves: {moves}")
        
        if current_state == goal_state:
            print(f"Max queue length: {max_queue_length}")
            return
        
        # Generate successors
        state_tuple = tuple(current_state)
        if state_tuple not in visited:
            visited.add(state_tuple)
            
            for successor in get_successors(current_state):
                if tuple(successor) not in visited:
                    h = get_manhattan_distance(successor)
                    g = moves + 1
                    f = g + h
                    heapq.heappush(pq, (f, g, successor, current_path))

def test_cases():
    """Test cases to verify the implementation."""

    print("7-tile puzzle:")
    solve([4,3,8,5,1,6,7,2,0])  # Valid 7-tile puzzle
    
    print("Duplicate numbers:")
    solve([1,2,2,3,4,5,6,0,0])  # Has duplicate 2
    
    print("\nNon-continuous integers:")
    solve([1,2,3,5,6,7,8,0,0])  # Missing 4
    
    print("\nSkipped integers:")
    solve([1,2,4,5,6,0,0,0,0])  # Missing 3

In [23]:
test_cases()

7-tile puzzle:
True
[4, 3, 8, 5, 1, 6, 7, 2, 0] h=10 moves: 0
[4, 3, 8, 5, 1, 0, 7, 2, 6] h=11 moves: 1
[4, 3, 8, 5, 1, 6, 7, 0, 2] h=11 moves: 1
[4, 3, 0, 5, 1, 8, 7, 2, 6] h=10 moves: 2
[4, 0, 3, 5, 1, 8, 7, 2, 6] h=9 moves: 3
[4, 1, 3, 5, 0, 8, 7, 2, 6] h=8 moves: 4
[4, 1, 3, 0, 5, 8, 7, 2, 6] h=7 moves: 5
[4, 1, 3, 5, 2, 8, 7, 0, 6] h=7 moves: 5
[4, 1, 3, 5, 8, 0, 7, 2, 6] h=7 moves: 5
[0, 1, 3, 4, 5, 8, 7, 2, 6] h=6 moves: 6
[4, 1, 3, 5, 8, 6, 7, 2, 0] h=6 moves: 6
[1, 0, 3, 4, 5, 8, 7, 2, 6] h=5 moves: 7
[4, 3, 8, 5, 0, 1, 7, 2, 6] h=12 moves: 2
[4, 3, 8, 5, 0, 6, 7, 1, 2] h=12 moves: 2
[4, 3, 8, 5, 1, 6, 0, 7, 2] h=12 moves: 2
[4, 3, 8, 0, 5, 1, 7, 2, 6] h=11 moves: 3
[4, 3, 8, 0, 5, 6, 7, 1, 2] h=11 moves: 3
[4, 3, 8, 5, 2, 1, 7, 0, 6] h=11 moves: 3
[0, 3, 8, 4, 5, 1, 7, 2, 6] h=10 moves: 4
[0, 3, 8, 4, 5, 6, 7, 1, 2] h=10 moves: 4
[0, 4, 3, 5, 1, 8, 7, 2, 6] h=10 moves: 4
[4, 1, 0, 5, 8, 3, 7, 2, 6] h=8 moves: 6
[4, 1, 3, 5, 2, 8, 0, 7, 6] h=8 moves: 6
[4, 1, 3, 5, 2, 8, 7, 6,

In [24]:
import heapq

def get_manhattan_distance(state, goal_state):
    """Calculate the Manhattan distance heuristic."""
    distance = 0
    size = 3  # 3x3 grid
    
    # Create position mappings for goal state
    goal_positions = {}
    for i, val in enumerate(goal_state):
        if val != 0:
            goal_positions[val] = (i // size, i % size)
    
    # Calculate Manhattan distance for each tile
    for i, val in enumerate(state):
        if val != 0:  # Skip empty spaces
            curr_pos = (i // size, i % size)
            goal_pos = goal_positions[val]
            distance += abs(curr_pos[0] - goal_pos[0]) + abs(curr_pos[1] - goal_pos[1])
    
    return distance

def get_successors(state):
    """Generate all possible successor states."""
    successors = []
    size = 3  # 3x3 grid
    
    # Find indices of empty spaces (0s)
    empty_indices = [i for i, val in enumerate(state) if val == 0]
    
    for empty_idx in empty_indices:
        row, col = empty_idx // size, empty_idx % size
        
        # Check all possible moves (up, down, left, right)
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in moves:
            new_row, new_col = row + dx, col + dy
            
            # Check if move is valid
            if 0 <= new_row < size and 0 <= new_col < size:
                new_idx = new_row * size + new_col
                # Make sure we're not swapping with another empty space
                if state[new_idx] != 0:
                    # Create new state with the move
                    new_state = list(state)
                    new_state[empty_idx], new_state[new_idx] = new_state[new_idx], new_state[empty_idx]
                    successors.append(new_state)
    
    return sorted(successors)

def print_succ(state):
    """Print all successor states with their heuristic values."""
    # Determine number of tiles and create goal state
    num_tiles = sum(1 for x in state if x != 0)
    goal_state = list(range(1, num_tiles + 1)) + [0] * (9 - num_tiles)
    
    # Generate and print successors
    successors = get_successors(state)
    for succ in successors:
        h = get_manhattan_distance(succ, goal_state)
        print(succ, f"h={h}")

def solve(state):
    """Solve the puzzle using A* search."""
    # Determine number of tiles and create goal state
    num_tiles = sum(1 for x in state if x != 0)
    goal_state = list(range(1, num_tiles + 1)) + [0] * (9 - num_tiles)
    
    # Initialize priority queue
    pq = []
    initial_h = get_manhattan_distance(state, goal_state)
    heapq.heappush(pq, (initial_h, 0, state, [], 0))  # (priority, moves, state, path, index)
    
    # Keep track of visited states to avoid cycles
    visited = set()
    visited.add(tuple(state))
    
    # Keep track of maximum queue length
    max_queue_length = 1
    
    while pq:
        max_queue_length = max(max_queue_length, len(pq))
        _, moves, current_state, path, _ = heapq.heappop(pq)
        
        # Check if we've reached the goal
        if current_state == goal_state:
            print("True")
            # Print solution path
            full_path = [state] + path + [current_state]
            for i, s in enumerate(full_path):
                h = get_manhattan_distance(s, goal_state)
                print(s, f"h={h} moves: {i}")
            print(f"Max queue length: {max_queue_length}")
            return
        
        # Generate successors
        successors = get_successors(current_state)
        for succ in successors:
            if tuple(succ) not in visited:
                visited.add(tuple(succ))
                h = get_manhattan_distance(succ, goal_state)
                g = moves + 1
                priority = g + h
                heapq.heappush(pq, (priority, g, succ, path + [succ], len(visited)))
    
    # If no solution is found
    print("False")

In [25]:
# Example usage:
state = [4, 3, 0, 5, 1, 6, 7, 2, 0]

# Print successors
print_succ(state)

# Solve puzzle
solve(state)

[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6
[4, 3, 0, 5, 1, 0, 7, 2, 6] h=8
[4, 3, 0, 5, 1, 6, 7, 0, 2] h=8
[4, 3, 6, 5, 1, 0, 7, 2, 0] h=8
True
[4, 3, 0, 5, 1, 6, 7, 2, 0] h=7 moves: 0
[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6 moves: 1
[4, 1, 3, 5, 0, 6, 7, 2, 0] h=5 moves: 2
[4, 1, 3, 0, 5, 6, 7, 2, 0] h=4 moves: 3
[0, 1, 3, 4, 5, 6, 7, 2, 0] h=3 moves: 4
[0, 1, 3, 4, 5, 0, 7, 2, 6] h=4 moves: 5
[0, 1, 3, 4, 0, 5, 7, 2, 6] h=5 moves: 6
[0, 1, 3, 4, 2, 5, 7, 0, 6] h=4 moves: 7
[1, 0, 3, 4, 2, 5, 7, 0, 6] h=3 moves: 8
[1, 2, 3, 4, 0, 5, 7, 0, 6] h=2 moves: 9
[1, 2, 3, 4, 5, 0, 7, 0, 6] h=1 moves: 10
[1, 2, 3, 4, 5, 6, 7, 0, 0] h=0 moves: 11
[1, 2, 3, 4, 5, 6, 7, 0, 0] h=0 moves: 12
Max queue length: 185


In [28]:
import heapq

def get_manhattan_distance(state):
    """Calculate Manhattan distance heuristic."""
    distance = 0
    for i in range(len(state)):
        if state[i] != 0:  # Skip empty tiles
            curr_row, curr_col = i // 3, i % 3
            goal_row, goal_col = (state[i] - 1) // 3, (state[i] - 1) % 3
            distance += abs(curr_row - goal_row) + abs(curr_col - goal_col)
    return distance

def get_successors(state):
    """Generate all valid successor states."""
    successors = []
    empty_positions = [i for i, val in enumerate(state) if val == 0]
    
    for empty_pos in empty_positions:
        row, col = empty_pos // 3, empty_pos % 3
        
        # Check all possible moves (up, down, left, right)
        for drow, dcol in [(-1,0), (1,0), (0,-1), (0,1)]:
            new_row, new_col = row + drow, col + dcol
            
            # Check if move is valid
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                # Only move if the position contains a tile (not 0)
                if state[new_pos] != 0:
                    new_state = list(state)
                    new_state[empty_pos], new_state[new_pos] = new_state[new_pos], new_state[empty_pos]
                    # Only add if this configuration hasn't been seen
                    if new_state not in successors:
                        successors.append(new_state)
    
    return sorted(successors)

def print_succ(state):
    """Print all successor states."""
    successors = get_successors(state)
    for succ in successors:
        h = get_manhattan_distance(succ)
        print(succ, f"h={h}")

def solve(state):
    """Solve the puzzle using A* search."""
    # Initialize data structures
    visited = set()
    pq = []
    initial_h = get_manhattan_distance(state)
    
    # Priority queue entries: (f, g, state, parent_index)
    # f = g + h (total cost)
    # g = number of moves
    # parent_index = index in visited list for reconstructing path
    heapq.heappush(pq, (initial_h, 0, state, -1))
    states_list = []  # List to store states for path reconstruction
    
    max_queue_length = 1
    
    while pq:
        max_queue_length = max(max_queue_length, len(pq))
        f, g, curr_state, parent_idx = heapq.heappop(pq)
        curr_tuple = tuple(curr_state)
        
        # Skip if we've seen this state
        if curr_tuple in visited:
            continue
            
        # Add current state to visited set and states list
        visited.add(curr_tuple)
        states_list.append((curr_state, parent_idx))
        curr_idx = len(states_list) - 1
        
        # Check if we've reached the goal
        if curr_state == [1,2,3,4,5,6,7,0,0]:
            print("True")
            
            # Reconstruct path
            path = []
            while curr_idx != -1:
                path.append(states_list[curr_idx][0])
                curr_idx = states_list[curr_idx][1]
            
            # Print solution path in reverse order
            path = path[::-1]
            for i, s in enumerate(path):
                h = get_manhattan_distance(s)
                print(s, f"h={h} moves: {i}")
                
            print(f"Max queue length: {max_queue_length}")
            return
            
        # Generate and process successors
        for succ in get_successors(curr_state):
            if tuple(succ) not in visited:
                h = get_manhattan_distance(succ)
                g_new = g + 1
                f_new = g_new + h
                heapq.heappush(pq, (f_new, g_new, succ, curr_idx))
    
    print("False")

In [30]:
# Test case from assignment
test_state = [4,3,0,5,1,6,7,2,0]
print("Successor states:")
print_succ(test_state)
print("\nSolution:")
solve(test_state)

Successor states:
[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6
[4, 3, 0, 5, 1, 0, 7, 2, 6] h=8
[4, 3, 0, 5, 1, 6, 7, 0, 2] h=8
[4, 3, 6, 5, 1, 0, 7, 2, 0] h=8

Solution:
True
[4, 3, 0, 5, 1, 6, 7, 2, 0] h=7 moves: 0
[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6 moves: 1
[4, 1, 3, 5, 0, 6, 7, 2, 0] h=5 moves: 2
[4, 1, 3, 0, 5, 6, 7, 2, 0] h=4 moves: 3
[0, 1, 3, 4, 5, 6, 7, 2, 0] h=3 moves: 4
[0, 1, 3, 4, 5, 0, 7, 2, 6] h=4 moves: 5
[0, 1, 3, 4, 0, 5, 7, 2, 6] h=5 moves: 6
[0, 1, 3, 4, 2, 5, 7, 0, 6] h=4 moves: 7
[1, 0, 3, 4, 2, 5, 7, 0, 6] h=3 moves: 8
[1, 2, 3, 4, 0, 5, 7, 0, 6] h=2 moves: 9
[1, 2, 3, 4, 5, 0, 7, 0, 6] h=1 moves: 10
[1, 2, 3, 4, 5, 6, 7, 0, 0] h=0 moves: 11
Max queue length: 218


In [40]:
import heapq
from typing import List, Set, Tuple

class PuzzleNode:
    def __init__(self, state: List[int], parent=None, g_cost=0):
        self.state = state
        self.parent = parent
        self.g_cost = g_cost  # Path cost
        self.h_cost = self.calculate_heuristic()  # Heuristic cost
        self.f_cost = self.g_cost + self.h_cost
        
    def __lt__(self, other):
        return self.f_cost < other.f_cost
    
    def calculate_heuristic(self) -> int:
        """Calculate Manhattan distance heuristic."""
        total_cost = 0
        for i, val in enumerate(self.state):
            if val != 0:  # Skip empty spaces
                # Calculate target position (bottom-up, left-right filling)
                target_row = (val - 1) // 3
                target_col = (val - 1) % 3
                current_row = i // 3
                current_col = i % 3
                total_cost += abs(target_row - current_row) + abs(target_col - current_col)
        return total_cost

def get_valid_moves(state: List[int]) -> List[Tuple[int, int]]:
    """Get all valid moves for the current state."""
    moves = []
    empty_positions = [i for i, val in enumerate(state) if val == 0]
    
    for empty_pos in empty_positions:
        row, col = empty_pos // 3, empty_pos % 3
        
        # Check all four directions
        directions = [
            (-1, 0),  # Up
            (1, 0),   # Down
            (0, -1),  # Left
            (0, 1)    # Right
        ]
        
        for dx, dy in directions:
            new_row, new_col = row + dx, col + dy
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                tile_pos = new_row * 3 + new_col
                # Only move if the position contains a tile (not empty)
                if state[tile_pos] != 0:
                    moves.append((empty_pos, tile_pos))
                    
    return moves

def apply_move(state: List[int], move: Tuple[int, int]) -> List[int]:
    """Apply a move to the state and return new state."""
    new_state = state.copy()
    empty_pos, tile_pos = move
    new_state[empty_pos], new_state[tile_pos] = new_state[tile_pos], new_state[empty_pos]
    return new_state

def is_goal_state(state: List[int]) -> bool:
    """Check if current state is goal state."""
    # Goal state should have numbers 1-7 in order followed by two zeros
    target = [1, 2, 3, 4, 5, 6, 7, 0, 0]
    return state == target

def solve_puzzle(initial_state: List[int]) -> List[List[int]]:
    """Solve the puzzle using A* search."""
    start_node = PuzzleNode(initial_state)
    frontier = [start_node]  # Priority queue
    explored = set()  # Set of explored states
    
    while frontier:
        current = heapq.heappop(frontier)
        
        # Convert current state to tuple for hashing
        state_tuple = tuple(current.state)
        
        if state_tuple in explored:
            continue
            
        if is_goal_state(current.state):
            # Reconstruct path
            path = []
            while current:
                path.append(current.state)
                current = current.parent
            return list(reversed(path))
            
        explored.add(state_tuple)
        
        # Generate and evaluate all possible moves
        for move in get_valid_moves(current.state):
            new_state = apply_move(current.state, move)
            if tuple(new_state) not in explored:
                new_node = PuzzleNode(
                    new_state,
                    parent=current,
                    g_cost=current.g_cost + 1
                )
                heapq.heappush(frontier, new_node)
    
    return []  # No solution found

def validate_input(state: List[int]) -> bool:
    """Validate the input state."""
    if len(state) != 9:
        return False
    # Check if numbers are 0-7
    valid_nums = set(range(8))
    return all(num in valid_nums for num in state) and state.count(0) == 2

def main():
    # Example usage
    initial_state = [4, 3, 0, 5, 1, 6, 7, 2, 0]
    
    if not validate_input(initial_state):
        print("Invalid input state")
        return
        
    solution = solve_puzzle(initial_state)
    
    if solution:
        print(f"Solution found in {len(solution)-1} moves:")
        for i, state in enumerate(solution):
            print(f"Step {i}:")
            for row in range(3):
                print(state[row*3:(row+3)*3])
            print()
    else:
        print("No solution found")

if __name__ == "__main__":
    main()

Solution found in 11 moves:
Step 0:
[4, 3, 0, 5, 1, 6, 7, 2, 0]
[5, 1, 6, 7, 2, 0]
[7, 2, 0]

Step 1:
[4, 0, 3, 5, 1, 6, 7, 2, 0]
[5, 1, 6, 7, 2, 0]
[7, 2, 0]

Step 2:
[4, 1, 3, 5, 0, 6, 7, 2, 0]
[5, 0, 6, 7, 2, 0]
[7, 2, 0]

Step 3:
[4, 1, 3, 0, 5, 6, 7, 2, 0]
[0, 5, 6, 7, 2, 0]
[7, 2, 0]

Step 4:
[4, 1, 3, 0, 5, 0, 7, 2, 6]
[0, 5, 0, 7, 2, 6]
[7, 2, 6]

Step 5:
[4, 1, 3, 0, 0, 5, 7, 2, 6]
[0, 0, 5, 7, 2, 6]
[7, 2, 6]

Step 6:
[4, 1, 3, 0, 2, 5, 7, 0, 6]
[0, 2, 5, 7, 0, 6]
[7, 0, 6]

Step 7:
[0, 1, 3, 4, 2, 5, 7, 0, 6]
[4, 2, 5, 7, 0, 6]
[7, 0, 6]

Step 8:
[1, 0, 3, 4, 2, 5, 7, 0, 6]
[4, 2, 5, 7, 0, 6]
[7, 0, 6]

Step 9:
[1, 2, 3, 4, 0, 5, 7, 0, 6]
[4, 0, 5, 7, 0, 6]
[7, 0, 6]

Step 10:
[1, 2, 3, 4, 5, 0, 7, 0, 6]
[4, 5, 0, 7, 0, 6]
[7, 0, 6]

Step 11:
[1, 2, 3, 4, 5, 6, 7, 0, 0]
[4, 5, 6, 7, 0, 0]
[7, 0, 0]



In [39]:
import heapq
from typing import List

def calculate_heuristic(state: List[int]) -> int:
    """Calculate sum of Manhattan distances to goal positions."""
    h = 0
    for i, val in enumerate(state):
        if val != 0:  # Don't count empty spaces
            # Calculate current position
            curr_row, curr_col = i // 3, i % 3
            # Calculate goal position (0-indexed)
            goal_row, goal_col = (val-1) // 3, (val-1) % 3
            h += abs(curr_row - goal_row) + abs(curr_col - goal_col)
    return h

def get_successors(state: List[int]) -> List[List[int]]:
    """Get all possible successor states."""
    successors = []
    # Find empty positions (index of 0s)
    empty_pos = [i for i, val in enumerate(state) if val == 0]
    
    for pos in empty_pos:
        row, col = pos // 3, pos % 3
        # Check all four directions
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            new_row, new_col = row + dx, col + dy
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                if state[new_pos] != 0:  # Only move non-empty tiles
                    new_state = state.copy()
                    new_state[pos], new_state[new_pos] = new_state[new_pos], new_state[new_pos]
                    successors.append(new_state)
    
    return sorted(successors)  # Sort in ascending order

def print_succ(state: List[int]) -> None:
    """Print all successor states and their heuristic values."""
    successors = get_successors(state)
    for succ in successors:
        print(succ, "h={}".format(calculate_heuristic(succ)))

def solve(state: List[int]) -> None:
    """Solve puzzle using A* search and print path to goal."""
    initial_h = calculate_heuristic(state)
    pq = []  # Priority queue: (f_cost, state, (g_cost, h_cost, parent_index))
    heapq.heappush(pq, (initial_h, state, (0, initial_h, -1)))
    
    visited = set()  # Track visited states
    states_list = []  # Store states for path reconstruction
    
    while pq:
        f, current_state, (g, h, parent_idx) = heapq.heappop(pq)
        state_tuple = tuple(current_state)
        
        if state_tuple in visited:
            continue
            
        states_list.append((current_state, parent_idx))
        current_idx = len(states_list) - 1
        visited.add(state_tuple)
        
        # Check if goal reached
        if all(current_state[i] == i+1 for i in range(7)) and current_state[7:] == [0,0]:
            # Reconstruct path
            path = []
            curr_idx = current_idx
            while curr_idx != -1:
                path.append(states_list[curr_idx][0])
                curr_idx = states_list[curr_idx][1]
            
            # Print path
            for state in reversed(path):
                print(state)
            return
        
        # Generate successors
        for succ_state in get_successors(current_state):
            if tuple(succ_state) not in visited:
                succ_h = calculate_heuristic(succ_state)
                succ_g = g + 1
                succ_f = succ_g + succ_h
                heapq.heappush(pq, (succ_f, succ_state, (succ_g, succ_h, current_idx)))
    
    print("No solution found")

# Example usage:
if __name__ == "__main__":
    # Test print_succ
    print("Testing print_succ:")
    print_succ([4, 3, 0, 5, 1, 6, 7, 2, 0])
    
    # Test solve
    print("\nTesting solve:")
    solve([4, 3, 0, 5, 1, 6, 7, 2, 0])

Testing print_succ:
[4, 3, 0, 5, 1, 6, 7, 2, 2] h=10
[4, 3, 0, 5, 1, 6, 7, 2, 6] h=8
[4, 3, 3, 5, 1, 6, 7, 2, 0] h=7
[4, 3, 6, 5, 1, 6, 7, 2, 0] h=8

Testing solve:
No solution found


In [41]:
def calculate_manhattan_distance(idx: int, value: int) -> int:
    """Calculate Manhattan distance from current position to goal position."""
    if value == 0:
        return 0
        
    # Current position
    current_row = idx // 3
    current_col = idx % 3
    
    # Goal position (value-1 since values start at 1)
    goal_row = (value - 1) // 3
    goal_col = (value - 1) % 3
    
    return abs(current_row - goal_row) + abs(current_col - goal_col)

def calculate_heuristic(state: List[int]) -> int:
    """Calculate heuristic value for given state."""
    return sum(calculate_manhattan_distance(i, val) 
              for i, val in enumerate(state) 
              if val != 0)

def get_successors(state: List[int]) -> List[List[int]]:
    """Get all possible successor states."""
    successors = []
    
    # Find indices of empty spaces (0s)
    empty_indices = [i for i, val in enumerate(state) if val == 0]
    
    # For each empty space
    for empty_idx in empty_indices:
        row = empty_idx // 3
        col = empty_idx % 3
        
        # Try all four directions
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            new_row = row + dr
            new_col = col + dc
            
            # Check if move is valid
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                swap_idx = new_row * 3 + new_col
                
                # Only swap if target position contains a tile (not 0)
                if state[swap_idx] != 0:
                    # Create new state with the move applied
                    new_state = state.copy()
                    new_state[empty_idx], new_state[swap_idx] = new_state[swap_idx], new_state[empty_idx]
                    successors.append(new_state)
    
    # Sort successors treating them as 9-digit numbers
    return sorted(successors)

def print_succ(state: List[int]) -> None:
    """Print successor states with their heuristic values."""
    for successor in get_successors(state):
        print(successor, "h=" + str(calculate_heuristic(successor)))

def solve(state: List[int]) -> None:
    """Solve the puzzle using A* search algorithm."""
    # Priority queue entries: (f_cost, state, (g_cost, h_cost, parent_index))
    initial_h = calculate_heuristic(state)
    pq = [(initial_h, state, (0, initial_h, -1))]
    visited = set()
    states = []  # List to store states for path reconstruction
    
    while pq:
        f, current_state, (g, h, parent) = heapq.heappop(pq)
        current_tuple = tuple(current_state)
        
        if current_tuple in visited:
            continue
            
        # Add state to visited and states list
        visited.add(current_tuple)
        current_index = len(states)
        states.append((current_state, parent))
        
        # Check if goal reached
        if current_state[:7] == [1,2,3,4,5,6,7] and current_state[7:] == [0,0]:
            # Reconstruct and print path
            path = []
            curr = current_index
            while curr != -1:
                path.append(states[curr][0])
                curr = states[curr][1]
            for state in reversed(path):
                print(state)
            return
            
        # Generate and enqueue successors
        for succ in get_successors(current_state):
            succ_tuple = tuple(succ)
            if succ_tuple not in visited:
                succ_h = calculate_heuristic(succ)
                succ_g = g + 1
                heapq.heappush(pq, (succ_g + succ_h, succ, (succ_g, succ_h, current_index)))

# Test cases
if __name__ == "__main__":
    # Example from instructions
    test_state = [2,5,1,4,0,6,7,0,3]
    print("Testing print_succ:")
    print_succ(test_state)

Testing print_succ:
[2, 0, 1, 4, 5, 6, 7, 0, 3] h=5
[2, 5, 1, 0, 4, 6, 7, 0, 3] h=7
[2, 5, 1, 4, 0, 6, 0, 7, 3] h=7
[2, 5, 1, 4, 0, 6, 7, 3, 0] h=7
[2, 5, 1, 4, 6, 0, 7, 0, 3] h=7


In [42]:
import heapq

def calculate_manhattan_distance(idx: int, value: int) -> int:
    """Calculate Manhattan distance from current position to goal position."""
    if value == 0:
        return 0
    current_row, current_col = idx // 3, idx % 3
    goal_row, goal_col = (value - 1) // 3, (value - 1) % 3
    return abs(current_row - goal_row) + abs(current_col - goal_col)

def calculate_heuristic(state) -> int:
    """Calculate total Manhattan distance heuristic."""
    return sum(calculate_manhattan_distance(i, val) 
              for i, val in enumerate(state) 
              if val != 0)

def get_successors(state) -> list:
    """Generate successor states."""
    successors = []
    empty_positions = [i for i, val in enumerate(state) if val == 0]
    
    for empty_pos in empty_positions:
        row, col = empty_pos // 3, empty_pos % 3
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                swap_pos = new_row * 3 + new_col
                if state[swap_pos] != 0:
                    new_state = state.copy()
                    new_state[empty_pos], new_state[swap_pos] = new_state[swap_pos], new_state[empty_pos]
                    successors.append(new_state)
    
    return sorted(successors)

def print_succ(state):
    """Print successor states with their heuristic values."""
    for succ in get_successors(state):
        print(succ, "h=" + str(calculate_heuristic(succ)))

def solve(state):
    """Solve the puzzle using A* search."""
    pq = []  # Priority queue: (cost, state, (g, h, parent_index))
    states_list = []  # List to store states for path reconstruction
    visited = set()
    max_queue_length = 0
    
    # Initial state
    h = calculate_heuristic(state)
    heapq.heappush(pq, (h, state, (0, h, -1)))
    
    while pq:
        max_queue_length = max(max_queue_length, len(pq))
        _, current_state, (g, h, parent_index) = heapq.heappop(pq)
        
        current_tuple = tuple(current_state)
        if current_tuple in visited:
            continue
            
        current_index = len(states_list)
        states_list.append((current_state, parent_index, g))
        visited.add(current_tuple)
        
        # Check if goal reached
        if current_state[:7] == [1,2,3,4,5,6,7] and current_state[7:] == [0,0]:
            print(True)
            
            # Reconstruct path
            path = []
            curr_index = current_index
            while curr_index != -1:
                state, parent_idx, moves = states_list[curr_index]
                path.append((state, calculate_heuristic(state), moves))
                curr_index = parent_idx
            
            # Print path
            for state, h_val, moves in reversed(path):
                print(state, f"h={h_val} moves: {moves}")
            
            print(f"Max queue length: {max_queue_length}")
            return
        
        # Generate and enqueue successors
        for succ in get_successors(current_state):
            succ_tuple = tuple(succ)
            if succ_tuple not in visited:
                succ_h = calculate_heuristic(succ)
                succ_g = g + 1
                heapq.heappush(pq, (succ_g + succ_h, succ, (succ_g, succ_h, current_index)))
    
    print(False)

In [43]:
print("\nTest 2: solve with example from instructions")
print("solve([4,3,0,5,1,6,7,2,0]):")
solve([4,3,0,5,1,6,7,2,0])


Test 2: solve with example from instructions
solve([4,3,0,5,1,6,7,2,0]):
True
[4, 3, 0, 5, 1, 6, 7, 2, 0] h=7 moves: 0
[4, 0, 3, 5, 1, 6, 7, 2, 0] h=6 moves: 1
[4, 1, 3, 5, 0, 6, 7, 2, 0] h=5 moves: 2
[4, 1, 3, 0, 5, 6, 7, 2, 0] h=4 moves: 3
[0, 1, 3, 4, 5, 6, 7, 2, 0] h=3 moves: 4
[0, 1, 3, 4, 5, 0, 7, 2, 6] h=4 moves: 5
[0, 1, 3, 4, 0, 5, 7, 2, 6] h=5 moves: 6
[0, 1, 3, 4, 2, 5, 7, 0, 6] h=4 moves: 7
[1, 0, 3, 4, 2, 5, 7, 0, 6] h=3 moves: 8
[1, 2, 3, 4, 0, 5, 7, 0, 6] h=2 moves: 9
[1, 2, 3, 4, 5, 0, 7, 0, 6] h=1 moves: 10
[1, 2, 3, 4, 5, 6, 7, 0, 0] h=0 moves: 11
Max queue length: 91
